# Open Shell Code tests


Broadly, there are three things we need to do make sure of before we can include the open shell code into the main repo.
1. Closed shell code is not affected in any way by the implementation of open shell.
2. Open shell code replicates closed shell results accurately.
3. Open shell results are accurate.

## 1.

We can check this using the existing tests for closed shell code.

## 2.

Overall
- Run closed shell experiemnts with both open and closed shell methods.
- Hamiltonians are numerically close

Localizers
- Active MOs should be consistent across un/restricted for closed shells.
- 

In [1]:
from nbed.driver import NbedDriver
from pathlib import Path

mol_filepath = Path("molecules/water.xyz").absolute()

args = {
    "geometry": str(mol_filepath),
    "n_active_atoms": 1,
    "basis": "STO-3G",
    "xc_functional": "b3lyp",
    "projector": "mu",
    "localization": "spade",
    "convergence": 1e-8,
    "savefile": None,
    "run_ccsd_emb": False,
    "run_fci_emb": False,
}

restric_driver = NbedDriver(
    geometry=args["geometry"],
    n_active_atoms=args["n_active_atoms"],
    basis=args["basis"],
    xc_functional=args["xc_functional"],
    projector=args["projector"],
    localization=args["localization"],
    convergence=args["convergence"],
    savefile=args["savefile"],
    run_ccsd_emb=args["run_ccsd_emb"],
    run_fci_emb=args["run_fci_emb"],
    force_unrestricted=False,
)

unrestric_driver = NbedDriver(
    geometry=args["geometry"],
    n_active_atoms=args["n_active_atoms"],
    basis=args["basis"],
    xc_functional=args["xc_functional"],
    projector=args["projector"],
    localization=args["localization"],
    convergence=args["convergence"],
    savefile=args["savefile"],
    run_ccsd_emb=args["run_ccsd_emb"],
    run_fci_emb=args["run_fci_emb"],
    force_unrestricted=True,
)

/home/mwilliams/anaconda3/envs/nbed/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'
/home/mwilliams/anaconda3/envs/nbed/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# q=0 S=0
## Full system Hamiltonians should match global FCI energy.

Note that we use the energy from the global fci and the Molecular Orbital coefficients from the KS. The fci object does not have the correct structure for us to make a hamiltonian from it but the underlying HF object which both the FCI and KS objects use to construct MOs should give very similar orbitals, such that our two methods of exact solution match.

In [2]:
from nbed.ham_builder import HamiltonianBuilder
from openfermion import QubitOperator, get_sparse_operator, count_qubits
import scipy as sp

# First check restricted
# This uses the spinorb_from_spatial function defined by openfermion
# Get FCI energy
rks = restric_driver._global_fci.e_tot - restric_driver._global_ks.energy_nuc()
print(rks)

# Get the Hamiltonian and diagonalize it
rbuilder = HamiltonianBuilder(restric_driver._global_ks, 0, "jordan_wigner")
rham = rbuilder.build()
rdiag, _ = sp.sparse.linalg.eigsh(get_sparse_operator(rham), k=1, which="SA")
print(rdiag[0])
# print(rham, rdiag)

-84.29484027700336
-84.29484027704291


The driver requires two h_core matrices, one for each spin, but there is only one in the KS object, so we need to double this!

In [3]:
# Second check unrestricted
# FCI energy
import numpy as np

uks = unrestric_driver._global_fci.e_tot - unrestric_driver._global_ks.energy_nuc()
print(uks)

# Create hamiltonian and diagonalize
old_hcore = unrestric_driver._global_ks.get_hcore()
unrestric_driver._global_ks.get_hcore = lambda *args: np.array([old_hcore, old_hcore])

ubuilder = HamiltonianBuilder(unrestric_driver._global_ks, 0, "jordan_wigner")

uham = ubuilder.build()
udiag, _ = sp.sparse.linalg.eigsh(get_sparse_operator(uham), k=1, which="SA")
print(udiag[0])

-84.2948402770033
-84.2948402770551


Now lets check with charge and spin not equal to zero!

## Charge = $\pm1$ and Spin = 1

In [4]:
# First lets do one with an electron missing.
driver = NbedDriver(
    geometry=args["geometry"],
    n_active_atoms=args["n_active_atoms"],
    basis=args["basis"],
    xc_functional=args["xc_functional"],
    projector=args["projector"],
    localization=args["localization"],
    convergence=args["convergence"],
    savefile=args["savefile"],
    run_ccsd_emb=args["run_ccsd_emb"],
    run_fci_emb=args["run_fci_emb"],
    force_unrestricted=True,
    charge=1,
    spin=1,
)

fci = driver._global_fci.e_tot - driver._global_ks.energy_nuc()
print(fci)

old_hcore = driver._global_ks.get_hcore()
driver._global_ks.get_hcore = lambda *args: np.array([old_hcore, old_hcore])

builder = HamiltonianBuilder(driver._global_ks, 0, "jordan_wigner")
ham = builder.build()
diag, _ = sp.sparse.linalg.eigsh(get_sparse_operator(ham), k=1, which="SA")
print(diag[0])

driver = NbedDriver(
    geometry=args["geometry"],
    n_active_atoms=args["n_active_atoms"],
    basis=args["basis"],
    xc_functional=args["xc_functional"],
    projector=args["projector"],
    localization=args["localization"],
    convergence=args["convergence"],
    savefile=args["savefile"],
    run_ccsd_emb=args["run_ccsd_emb"],
    run_fci_emb=args["run_fci_emb"],
    force_unrestricted=True,
    charge=-1,
    spin=1,
)

fci = driver._global_fci.e_tot - driver._global_ks.energy_nuc()
print(fci)

old_hcore = driver._global_ks.get_hcore()
driver._global_ks.get_hcore = lambda *args: np.array([old_hcore, old_hcore])

builder = HamiltonianBuilder(driver._global_ks, 0, "jordan_wigner")
ham = builder.build()
diag, _ = sp.sparse.linalg.eigsh(get_sparse_operator(ham), k=1, which="SA")
print(diag[0])

-83.97614236525206
-84.5711511116487
-83.68394377786765
-84.29840643100366


### FCI and diagonalisation results match for systems with closed shells and no spin, however the results __don't match for charge and spin $\ne$ 0__

## Charge = $\pm2$ and spin = 0

In [5]:
# Here's another one with an extra electron
driver = NbedDriver(
    geometry=args["geometry"],
    n_active_atoms=args["n_active_atoms"],
    basis=args["basis"],
    xc_functional=args["xc_functional"],
    projector=args["projector"],
    localization=args["localization"],
    convergence=args["convergence"],
    savefile=args["savefile"],
    run_ccsd_emb=args["run_ccsd_emb"],
    run_fci_emb=args["run_fci_emb"],
    force_unrestricted=True,
    charge=-2,
    spin=0,
)

fci = driver._global_fci.e_tot - driver._global_ks.energy_nuc()
print(fci)

old_hcore = driver._global_ks.get_hcore()
driver._global_ks.get_hcore = lambda *args: np.array([old_hcore, old_hcore])

builder = HamiltonianBuilder(driver._global_ks, 0, "jordan_wigner")
ham = builder.build()
diag, _ = sp.sparse.linalg.eigsh(get_sparse_operator(ham), k=1, which="SA")
print(diag[0])

# Here's another one with an extra electron
driver = NbedDriver(
    geometry=args["geometry"],
    n_active_atoms=args["n_active_atoms"],
    basis=args["basis"],
    xc_functional=args["xc_functional"],
    projector=args["projector"],
    localization=args["localization"],
    convergence=args["convergence"],
    savefile=args["savefile"],
    run_ccsd_emb=args["run_ccsd_emb"],
    run_fci_emb=args["run_fci_emb"],
    force_unrestricted=True,
    charge=2,
    spin=0,
)

fci = driver._global_fci.e_tot - driver._global_ks.energy_nuc()
print(fci)

old_hcore = driver._global_ks.get_hcore()
driver._global_ks.get_hcore = lambda *args: np.array([old_hcore, old_hcore])

builder = HamiltonianBuilder(driver._global_ks, 0, "jordan_wigner")
ham = builder.build()
diag, _ = sp.sparse.linalg.eigsh(get_sparse_operator(ham), k=1, which="SA")
print(diag[0])

-82.49664870547343
-85.67134914280695
-83.01046980051932
-84.98385690428752


From the above result we can see that the addition of a non-zero spin results in a significant increase (1E-4) in system energy according to our code, whereas the FCI result gives only 3E-11 change.

## First lets fix for q=2, S=0 so that only one factor has to be accounted for.

One body is straightforward, so let's check the two body.

$$\mathbb{1^{\sigma}_{pq}} = \bra{p}h_{pq}\ket{q}$$

$$\mathbb{2^{\sigma}_{pqrs}} = $$

In [6]:
from pyscf import ao2mo, scf, dft


def _two_body_integrals(c_matrix_active, scf_method) -> np.ndarray:
    """Get the two electron integrals."""

    if isinstance(scf_method, (scf.uhf.UHF, dft.uks.UKS)):
        n_orbs_alpha = c_matrix_active[0].shape[1]
        n_orbs_beta = c_matrix_active[1].shape[1]

        two_body_compressed_alpha = ao2mo.kernel(scf_method.mol, c_matrix_active[0])
        two_body_compressed_beta = ao2mo.kernel(scf_method.mol, c_matrix_active[1])

        # get electron repulsion integrals
        eri_alpha = ao2mo.restore(
            1, two_body_compressed_alpha, n_orbs_alpha
        )  # no permutation symmetry
        eri_beta = ao2mo.restore(1, two_body_compressed_beta, n_orbs_beta)

        # Openfermion uses physicist notation whereas pyscf uses chemists
        two_body_integrals_alpha = np.asarray(
            eri_alpha.transpose(0, 2, 3, 1), order="C"
        )
        two_body_integrals_beta = np.asarray(eri_beta.transpose(0, 2, 3, 1), order="C")

        two_body_integrals = np.array(
            [two_body_integrals_alpha, two_body_integrals_beta]
        )

    else:
        n_orbs = c_matrix_active.shape[1]

        two_body_compressed = ao2mo.kernel(scf_method.mol, c_matrix_active)

        # get electron repulsion integrals
        eri = ao2mo.restore(1, two_body_compressed, n_orbs)  # no permutation symmetry

        # Openfermion uses physicist notation whereas pyscf uses chemists
        two_body_integrals = np.asarray(eri.transpose(0, 2, 3, 1), order="C")

    return two_body_integrals

# Plan
- [] Recreate the correct results for [q=0 S=0](#q0-s0) explicitly
- [] Show extend to charge =2 and spin = 0

In [7]:
new_coeffs = np.array(
    [unrestric_driver._global_ks.mo_coeff[0], unrestric_driver._global_ks.mo_coeff[1]]
)

In [8]:
from pyscf import ao2mo

n_orbs_alpha = new_coeffs[0].shape[1]
n_orbs_beta = new_coeffs[1].shape[1]
print(f"{n_orbs_alpha=} {n_orbs_beta=}")

one_body_alpha = (
    new_coeffs[0].T @ unrestric_driver._global_ks.get_hcore()[0] @ new_coeffs[0]
)
one_body_beta = (
    new_coeffs[1].T @ unrestric_driver._global_ks.get_hcore()[1] @ new_coeffs[1]
)

# Whats the correct thing to do when n_orbs_alpha != n_orbs_beta?
# It will be possible, especially for spade if the gradient does not change quickly
eri_aaaa = ao2mo.restore(
    1,
    ao2mo.kernel(
        unrestric_driver._global_ks.mol,
        (new_coeffs[0], new_coeffs[0], new_coeffs[0], new_coeffs[0]),
    ),
    n_orbs_alpha,
)
eri_bbbb = ao2mo.restore(
    1,
    ao2mo.kernel(
        unrestric_driver._global_ks.mol,
        (new_coeffs[1], new_coeffs[1], new_coeffs[1], new_coeffs[1]),
    ),
    n_orbs_beta,
)
eri_abba = ao2mo.restore(
    1,
    ao2mo.kernel(
        unrestric_driver._global_ks.mol,
        (new_coeffs[0], new_coeffs[1], new_coeffs[1], new_coeffs[0]),
    ),
    n_orbs_alpha,
)
eri_baab = ao2mo.restore(
    1,
    ao2mo.kernel(
        unrestric_driver._global_ks.mol,
        (new_coeffs[1], new_coeffs[0], new_coeffs[0], new_coeffs[1]),
    ),
    n_orbs_beta,
)
eri_abab = ao2mo.restore(
    1,
    ao2mo.kernel(
        unrestric_driver._global_ks.mol,
        (new_coeffs[0], new_coeffs[1], new_coeffs[0], new_coeffs[1]),
    ),
    n_orbs_alpha,
)
eri_baba = ao2mo.restore(
    1,
    ao2mo.kernel(
        unrestric_driver._global_ks.mol,
        (new_coeffs[1], new_coeffs[0], new_coeffs[1], new_coeffs[0]),
    ),
    n_orbs_beta,
)

eri_aaaa = np.einsum("ijkl -> iklj", eri_aaaa)
eri_bbbb = np.einsum("ijkl -> iklj", eri_bbbb)
eri_abba = np.einsum("ijkl -> iklj", eri_abba)
eri_baab = np.einsum("ijkl -> iklj", eri_baab)
eri_abab = np.einsum("ijkl -> iklj", eri_abab)
eri_baba = np.einsum("ijkl -> iklj", eri_baba)


# eri_aaaa = np.asarray(
#                 eri_aaaa.transpose(0, 2, 3, 1), order="C"
#             )
# eri_bbbb = np.asarray(
#                 eri_bbbb.transpose(0, 2, 3, 1), order="C"
#             )
# eri_abba = np.asarray(
#                 eri_abba.transpose(0, 2, 3, 1), order="C"
#             )
# eri_baab = np.asarray(
#                 eri_baab.transpose(0, 2, 3, 1), order="C"
#             )
# eri_abab = np.asarray(
#                 eri_abab.transpose(0, 2, 3, 1), order="C"
#             )
# eri_baba = np.asarray(
#                 eri_baba.transpose(0, 2, 3, 1), order="C"
#             )

n_orbs_alpha=7 n_orbs_beta=7


In [9]:
eri_alpha = ao2mo.restore(
    1, ao2mo.kernel(unrestric_driver._global_ks.mol, new_coeffs[0]), n_orbs_alpha
)
eri_beta = ao2mo.restore(
    1, ao2mo.kernel(unrestric_driver._global_ks.mol, new_coeffs[1]), n_orbs_beta
)
eri_alpha = np.asarray(eri_alpha.transpose(0, 2, 3, 1), order="C")
eri_beta = np.asarray(eri_beta.transpose(0, 2, 3, 1), order="C")

The one and two body integrals between nbed code and explicit should match

In [10]:
(
    np.allclose(ubuilder._one_body_integrals[0], one_body_alpha),
    np.allclose(ubuilder._two_body_integrals[0], eri_alpha),
    np.allclose(ubuilder._one_body_integrals[1], one_body_beta),
    np.allclose(ubuilder._two_body_integrals[1], eri_beta),
    np.allclose(eri_aaaa, eri_alpha),
    np.allclose(eri_bbbb, eri_beta),
)

(True, True, True, True, True, True)

This code replicates the results of the closed shell code correctly

For molecular orbitals $p,q,r,s$, there are 16 coefficients corresponding to spins. Thankfully only 4 of them are non-zero:
$\braket{p_\alpha q_\beta|r_\gamma s_\delta} \ne 0$ only when $\alpha =\delta$ and $\beta = \gamma$

In [11]:
from openfermion.config import EQ_TOLERANCE

n_spin_orbs = n_orbs_alpha + n_orbs_beta
eri_mo_basis = np.zeros((n_spin_orbs, n_spin_orbs, n_spin_orbs, n_spin_orbs))
one_body_tensor = np.zeros((n_spin_orbs, n_spin_orbs))
for p in range(n_orbs_alpha):
    for q in range(n_orbs_alpha):
        # up,up
        one_body_tensor[2 * p, 2 * q] = one_body_alpha[p, q]

        # down,down
        one_body_tensor[2 * p + 1, 2 * q + 1] = one_body_beta[p, q]

        for r in range(n_orbs_alpha):
            for s in range(n_orbs_alpha):
                # pg 82 Szabo
                #  up down up down (physics notation)
                eri_mo_basis[2 * p, 2 * q + 1, 2 * r, 2 * s + 1] = eri_abab[p, q, r, s]
                # down up down up  (physics notation)
                eri_mo_basis[2 * p + 1, 2 * q, 2 * r + 1, 2 * s] = eri_baba[p, q, r, s]

                # up,up,up,up
                eri_mo_basis[2 * p, 2 * q, 2 * r, 2 * s] = eri_aaaa[p, q, r, s]
                # down,down, down, down
                eri_mo_basis[2 * p + 1, 2 * q + 1, 2 * r + 1, 2 * s + 1] = eri_bbbb[
                    p, q, r, s
                ]

one_body_tensor[np.absolute(one_body_tensor) < EQ_TOLERANCE] = 0.0
eri_mo_basis[np.absolute(eri_mo_basis) < EQ_TOLERANCE] = 0.0

In [12]:
from openfermion import InteractionOperator

fermionic_ham = InteractionOperator(0, one_body_tensor, 0.5 * eri_mo_basis)
explicit_ham = ubuilder._qubit_transform("jordan_wigner", fermionic_ham)
len(explicit_ham.terms), len(uham.terms)
diag, _ = sp.sparse.linalg.eigsh(get_sparse_operator(explicit_ham), k=1, which="SA")
print(diag)

[-122.60810052]


In [13]:
one_e, two_e = ubuilder._unrestricted_spinorb_from_spatial(
    np.array([one_body_alpha, one_body_beta]), np.array([eri_aaaa, eri_bbbb])
)

In [126]:
np.allclose(one_e, one_body_tensor), np.allclose(two_e, eri_mo_basis)

((14, 14, 14, 14), (14, 14, 14, 14))

In [ ]:
from openfermion.chem.molecular_data import spinorb_from_spatial


## 3.

This is the trickiest bit
- We can first check that the general behaviour is correct, what do we expect for the +1 and -1 ions of a molecule?
- Check results against physical limits, for instance if any are below the ground state these need to be explained.
- Find published energies of some example ions and compare to these.